In [1]:
from tensorflow.keras import layers
import gymnasium as gym
#from gym import spaces
import tensorflow as tf
import numpy as np
import scanpy as sc
import cellrank as cr
import pandas as pd
#import gym

Global seed set to 0
/Users/brendamelano/miniconda3/envs/scvi_env/lib/python3.11/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/Users/brendamelano/miniconda3/envs/scvi_env/lib/python3.11/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [ ]:
# Training the agent with DQN
#from stable_baseline3 import DQN 

In [ ]:
action_df = pd.read_csv('/Users/brendamelano/Desktop/RL_for_reprogramming/action_df.csv')

In [ ]:
class LatentSpaceEnv(gym.Env):
    def __init__(self, action_df, target_point):
        super(LatentSpaceEnv, self).__init__()

        # The action_df describes how each perturbation affects each latent variable
        self.action_df = action_df
        self.action_space = gym.spaces.Discrete(len(action_df))

        # Low and high should be determined by the min and max of the latent variables in the embedding
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(6,), dtype=np.float32)

        # It's important for this to be random for the learning phase, but will be set to cluster centers late
        self.current_state = np.random.rand(6)

        # Needs to be set a the epicenter of a certain cluster
        self.target_point = target_point

    def step(self, action):
        
        # Apply action to current_state
        action_effect = self.action_df.iloc[action].values
        self.current_state += action_effect

        # Calculate Euclidean distance to target_point as the reward
        distance = np.linalg.norm(self.current_state - self.target_point)
        reward = -distance  # Negative reward, lower distance is better

        # Compute the new position of the cell in the embedding
        #distances = np.linalg.norm(modulated_embedding - self.current_observation, axis=1)
        #new_observation = modulated_embedding[np.argmin(distances)]
        
        # Calculate the reward based on the probability of absorption
        # This can be used at a later point. Use distance first
        #reward = self.transition_matrix[self.current_observation, new_observation]

        # Check if the target is reached or sufficiently close
        done = distance < 0.01

        # Check if the next state is the terminal state
        #done = (new_observation == self.transition_matrix.shape[0] - 1)

        # Update the state of the environment 
        self.current_observation = new_observation

        self.reward_range = (0,1)
        
        return self.current_state, reward, done, {}

    def reset(self):
        self.current_state = np.random.rand(6)
        return self.current_state

In [ ]:
class SimpleDQN:
    def __init__(self, observation_space, action_space, alpha, gamma, epsilon):
        self.observation_space = observation_space
        self.action_space = action_space
        self.model = self.create_model()
        self.alpha = alpha  # learning rate
        self.gamma = gamma  # discount factor
        self.epsilon = epsilon  # exploration rate
        self.Q = np.zeros((11, 2))

    def create_model(self):
        model = tf.keras.Sequential()
        model.add(layers.Input(shape=(self.observation_space.shape[0],)))
        model.add(layers.Dense(256, activation='relu'))
        model.add(layers.Dense(256, activation='relu'))
        model.add(layers.Dense(self.action_space.n, activation='linear'))
        model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse')
        return model

    def choose_action(self, state):
        state = np.reshape(state, [1, self.observation_space.shape[0]])
        action_values = self.model.predict(state)
        return np.argmax(action_values[0])

    def choose_action(self, state):
        if np.random.uniform() < self.epsilon:
            action = np.random.choice([-1, 1])
        else:
            action = np.argmax(self.Q[state])
        return action

    def learn(self, state, action, reward, next_state):
        # Implement learning process
        pass

    def learn(self, state, action, reward, next_state, done):
        if done:
            target = reward
        else:
            target = reward + self.gamma * np.max(self.Q[next_state])
            self.Q[state, action] += self.alpha * (target - self.Q[state, action])
        
    def get_perturbations_to_target(self, initial_state, target_state, max_steps=100): 
        current_state = initial_state 
        perturbations = [] 
        for _ in range(max_steps): 
            action = self.choose_action(current_state) 
            next_state = self.apply_action(current_state, action) 
            perturbations.append(action) 
            
            if np.allclose(next_state, target_state, atol=1e-2): # You can adjust the tolerance break current_state = next_state return perturbations
                break

            current_state = next_state

        return perturbations

In [ ]:
# Creating the environment based on the class that I built
env = EmbeddingEnv(embedding_file = "/Users/brendamelano/Desktop/svae_latent_array_152.csv", transition_matrix = "/Users/brendamelano/Desktop/svae_abs_probs_152.csv")



In [ ]:
# Define the Q-learning agent
q_agent = DQN('M1pPolicy', env, learning_rate = 0.001, buffer_size = 50000, verbose = 1)


# Train the agent 
# Does not converge, just trains for specified number of episodes/steps
total_timesteps = 50000
q_agent.learn(total_timesteps = total_steps)


# Resetting the environment 
obs = env.reset()

# 
for i in range(10):
    done = False
    while not done:
        action, _states = q_agent.predict(obs, deterministic = True)
        obs, rewardm done, info = env.step(action)
    print(f"Episode {i+1} finished with reward {reward}")

In [18]:
# Initialize the environment with target point for a specific cluster
target_point = np.random.rand(6)

# Defining the environment
env = LatentSpaceEnv(action_df, target_point)

In [23]:
# Initialize the DQN model
model = SimpleDQN(env.observation_space, env.action_space)


In [ ]:
# Defining the number of training episodes
num_episodes = 10
for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0
    done = False

    while not done:
        action = model.choose_action(state)
        new_state, reward, done, _ = env.step(action)
        model.learn(state, action, reward, new_state)
        state = new_state
        total_reward += reward

    print(f"Episode: {episode}, Total Reward: {total_reward}")